In [12]:
import cv2
import numpy as np

In [13]:
import os
import json
import math

# For container dataset
x_dir = '../container_dataset/container_dataset/train_images/'
y_dir = '../container_dataset/container_dataset/train_labels/'

data = []

for filename in os.listdir(y_dir):
    f = os.path.join(y_dir, filename)

    if not os.path.isfile(f):
        continue

    label_json = open(f)
    label_dict = json.load(label_json)

    labels_list = [(i['properties']['object_imcoords'], i['properties']['object_angle']) for i in label_dict['features']]
    property_list = []
    coords_list = []

    for coords, angle in labels_list:
        coords = [float(k) for k in coords.split(',')]
        # print(coords)

        side_len1 = abs(coords[0] - coords[2])**2 + abs(coords[1] - coords[3])**2
        side_len1 = math.sqrt(side_len1)
        side_len2 = abs(coords[0] - coords[6])**2 + abs(coords[1] - coords[7])**2
        side_len2 = math.sqrt(side_len2)

        mid_x = (coords[0] + coords[4]) / 2
        mid_y = (coords[1] + coords[5]) / 2

        property_list.append((side_len1, side_len2, mid_x, mid_y, angle))
        coords_list.append(coords)

    data.append((os.path.join(f), property_list, coords_list))

In [15]:
img_data = []

for data_point in data:
    filename = data_point[0][:-4] + 'png'
    img_data.append((filename, data_point[1], data_point[2]))

In [16]:
import torch

class property_Dataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self._data_list = data
    
    def __len__(self):
        return len(self._data_list)

    def __getitem__(self, index):
        return (self._data_list[index][0], self._data_list[index][1])
    
class coords_Dataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self._data_list = data
    
    def __len__(self):
        return len(self._data_list)
    
    def __getitem__(self, index):
        return (self._data_list[index[0]], self._data_list[index][2])

In [20]:
import random

random.shuffle(img_data)

validation_ratio = 0.1
testing_ratio = 0.1

data_count = len(img_data)
validation_thresh = int(data_count * (1 - (validation_ratio + testing_ratio)))
testing_thresh = int(data_count * (1 - testing_ratio))

train_data = img_data[:validation_thresh]
val_data = img_data[validation_thresh:testing_thresh]
test_data = img_data[testing_thresh:]

print("Training Data", len(train_data), "samples initialized...")
print("Validation Data", len(val_data), "samples initialized...")
print("Testing Data", len(test_data), "samples initialized...")

Training Data 217 samples initialized...
Validation Data 27 samples initialized...
Testing Data 28 samples initialized...
